# Soportes Tickets

Si ejecuta en **VSCode** instale `pip install python-dotenv requests pandas ipykernel openpyxl`

Si ejecuta **Fedora** con **Jupyter Notebook** en el navegador instale `pip install -U python-dotenv openpyxl`

Debe tener una archivo `.env` donde guarde de forma segura su USERNAME y PASSWORD.

In [ ]:
# Cargar paquetes
import os
import pandas as pd
import requests

In [ ]:
# Cargar variables de entorno en .env
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# Hola a Plataforma Web API OAuth2
# HOST = "https://plataforma-web-api-oauth2.justiciadigital.gob.mx/"
HOST = "http://127.0.0.1:8002"
respuesta = requests.get(HOST)
print(respuesta.status_code)
print(respuesta.text)

In [ ]:
# Cargar username/password
parametros = {
    "username": os.getenv("USERNAME"),
    "password": os.getenv("PASSWORD"),
}
print(parametros["username"], parametros["password"][:4] + "...")

In [ ]:
# Solicitar token
cabeceras_para_autentificar = {'content-type': "application/x-www-form-urlencoded"}
respuesta = requests.post(f"{HOST}/token", data=parametros, headers=cabeceras_para_autentificar)
print(respuesta.status_code)

In [ ]:
# Definir cabeceras_para_api
cabeceras_para_api = None
if respuesta.status_code == 200:
    token = respuesta.json()['access_token']
    cabeceras_para_api = {'Authorization': 'Bearer ' + token}
print(cabeceras_para_api)

In [ ]:
# Función consultar
def consultar(ruta: str, parametros: dict = None):
    """Consultar la API entrega dos valores: listado/registro, total"""
    try:
        respuesta = requests.get(f"{HOST}/v1/{ruta}", headers=cabeceras_para_api, params=parametros, timeout=12)
    except requests.exceptions.RequestException as error:
        raise Exception("Parece que hay un error en la consulta: " + str(error))
    if respuesta.status_code != 200:
        raise requests.HTTPError("Parece que ya caducó el token")
    datos = respuesta.json()
    if "items" in datos:
        return pd.json_normalize(datos["items"]), datos["total"]
    return pd.json_normalize(datos), 1

In [ ]:
# Consultar totales de tickets por oficina y por categoria
estado = "TERMINADO"
parametros = {
    "creado_desde": "2022-02-28", 
    "creado_hasta": "2022-03-04", 
    "estado": estado, 
    "size": 200,
}
soportes, total = consultar("soportes_tickets/totales", parametros=parametros)
soportes.tail(8)

In [ ]:
# Cambiar el tipo de las columnas a category
soportes.distrito_clave = soportes.distrito_clave.astype("category")
soportes.soporte_categoria_nombre = soportes.soporte_categoria_nombre.astype("category")
soportes.info()

In [ ]:
# Pivot table
reporte = soportes.pivot_table(
    index=["soporte_categoria_nombre"], 
    columns=["distrito_clave"], 
    values="cantidad",
)
reporte.head(8)

In [ ]:
# Guardar reporte
from datetime import date
import openpyxl
hoy_str = date.today().strftime('%Y-%m-%d')
reporte.to_excel(f"reporte-{estado.lower()}-{hoy_str}.xlsx")